In [1]:
import sys
import os
sys.path.append('/workspace/persistent/Projects/Japanese-pronounce-project/lib')

In [2]:
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import json
from pydub import AudioSegment
import tempfile
import librosa
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import JvsDataset
import WordTimestamps


In [3]:
plt.rcParams["font.family"] = 'TakaoPGothic'
plt.rcParams["font.size"] = '14'

In [4]:
# file_name = ""
audio_flienames = [
    "/workspace/persistent/japanese project/jvs_ver1/jvs001/parallel100/wav24kHz16bit/VOICEACTRESS100_001.wav",
    "/workspace/persistent/japanese project/jvs_ver1/jvs002/parallel100/wav24kHz16bit/VOICEACTRESS100_001.wav",
    # "/workspace/persistent/japanese project/ohayo.wav",
    # "/workspace/persistent/japanese project/01-01-rec.m4a",
]

jvs_dataset = JvsDataset.JvsDataset(root="/workspace/persistent/datasets/jvs_ver1")
recognizer = WordTimestamps.Recognizer()

Reading your vosk model '/workspace/persistent/Projects/Japanese-pronounce-project/lib/vosk-model-ja-0.22'...


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 1 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 2 orphan components.
LOG (VoskAPI:Collapse():nnet-utils.cc:1488) Added 1 components, removed 2
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /workspace/persistent/Projects/Japanese-pronounce-project/lib/vosk-model-ja-0.22/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279) Loading HCLG from /workspace/persistent/Projects/Japanese-pronounce-project/lib/vosk-model-ja-0.22/graph/HCLG.fst
LOG (VoskAPI:ReadDataFiles():model.cc:294) Loading words from /workspace/persisten

'/workspace/persistent/Projects/Japanese-pronounce-project/lib/vosk-model-ja-0.22' model was successfully read


In [ ]:
recognizer.recognize(audio_flienames[0])

In [ ]:
from pydub import AudioSegment


speaker_1 = 1
speaker_2 = 2
parallel = 31

r1 = WordTimestamps.RecognizerResult(recognizer.recognize(jvs_dataset.get_audio_file_path(speaker_1, parallel)))
r2 = WordTimestamps.RecognizerResult(recognizer.recognize(jvs_dataset.get_audio_file_path(speaker_2, parallel)))

# 1 say word1
word_1 = 1
mfcc_1_word = r1[word_1].word
mfcc_1 = WordTimestamps.get_mfcc_of_clip(jvs_dataset.get_audio_file_path(speaker_1, parallel), r1[word_1].start, r1[word_1].end)
# 2 say word1
mfcc_2_word = r2[word_1].word
mfcc_2 = WordTimestamps.get_mfcc_of_clip(jvs_dataset.get_audio_file_path(speaker_2, parallel), r2[word_1].start, r2[word_1].end)
# 2 say word2
word_2 = 2
mfcc_3_word = r2[word_2].word
mfcc_3 = WordTimestamps.get_mfcc_of_clip(jvs_dataset.get_audio_file_path(speaker_2, parallel), r2[word_2].start, r2[word_2].end)

print("speaker1:{} <-> speaker2:{} : distance = {}".format(mfcc_1_word, mfcc_2_word, WordTimestamps.get_distance_of_mfcc(mfcc_1, mfcc_2)))
print("speaker1:{} <-> speaker2:{} : distance = {}".format(mfcc_1_word, mfcc_3_word, WordTimestamps.get_distance_of_mfcc(mfcc_1, mfcc_3)))
print("speaker2:{} <-> speaker2:{} : distance = {}".format(mfcc_2_word, mfcc_3_word, WordTimestamps.get_distance_of_mfcc(mfcc_2, mfcc_3)))

In [ ]:
import pandas as pd


# ignore UserWarning
import warnings
warnings.filterwarnings("ignore")


parallel = 31

data = []
r_all = []

for speaker in range(1, 11):
    r = WordTimestamps.RecognizerResult(recognizer.recognize(jvs_dataset.get_audio_file_path(speaker, parallel)))
    r_all.append(r)
    print("speaker {:02d} says parallel {}: len({})".format(speaker, parallel, len(r.result)), r.transcript)
    
    # mfccs = pd.DataFrame(np.array([ pd.DataFrame(i.ravel()) for i in get_mfcc_of_result(jvs_dataset.get_audio_file_path(speaker, parallel), r) ]))
    mfccs = pd.DataFrame(np.array([ pd.DataFrame(i.ravel()) for i in WordTimestamps.get_audios_of_result(jvs_dataset.get_audio_file_path(speaker, parallel), r) ])) # not using mfcc
    data.append(mfccs)

# all

In [ ]:

def plot_heatmap(data, speaker_1, speaker_2):
    matrix = []
    m = []

    for i in data[speaker_1][0]:
        for j in data[speaker_2][0]:
            m.append(WordTimestamps.get_distance_of_mfcc(i.T, j.T))

        matrix.append(m)
        m = []

    df = pd.DataFrame(matrix, index = [ r_all[speaker_1].result[i].word for i in range(r_all[speaker_1].length)],
                    columns = [ r_all[speaker_2].result[i].word for i in range(r_all[speaker_2].length)]).astype(int)

    plt.figure(figsize = (15,12))
    ax = plt.axes()
    ax.set_title('Speaker:{}  vs Speaker:{}'.format(speaker_1, speaker_2), fontsize=20)
    sn.heatmap(df, annot=True, cmap='Blues', robust=False, fmt='d', linewidths=.5)

    # output
    # plt.show()
    plt.savefig('./heatmaps/{}-{}-heatmap.png'.format(speaker_1, speaker_2))


In [ ]:
plot_heatmap(data, 0, 0)

In [ ]:
for i, j in np.ndindex((10 ,10)):
    plot_heatmap(data, i, j)
    print('print {}-{}'.format(i, j))